In [183]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.sparse import lil_matrix
import scipy.spatial.distance

In [184]:
pgadata = pd.read_csv('PGATOUR_data2.csv')
pgadata.head()

,Player,EVENTS_PLAYED,POINTS,NUMBER_OF_WINS,NUMBER_OF_TOP_Tens,POINTS_BEHIND_LEAD,ROUNDS_PLAYED,SG_PUTTING_PER_ROUND,TOTAL_SG:PUTTING,MEASURED_ROUNDS,...,AVG_CARRY_DISTANCE,LONGEST_CARRY_DISTANCE,SHORTEST_CARRY_DISTANCE,AVG_SCORE,TOTAL_STROKES,TOTAL_ROUNDS,MAKES_BOGEY%,BOGEYS_MADE,HOLES_PLAYED,AGE
0,Aaron Baddeley,27,342,0,1,5275.0,84,-0.062,-4.064,66,...,278.2,312.6,237.1,70.850,5518,78,16.24,228,1404,37
1,Aaron Wise,27,1498,1,4,4119.0,82,0.245,17.134,70,...,282.5,314.6,255.7,70.276,5888,84,16.53,250,1512,22
2,Abraham Ancer,29,909,0,5,4708.0,94,-0.052,-4.076,79,...,275.1,305.8,223.5,70.659,6745,96,14.76,255,1728,27
3,Adam Hadwin,24,1054,0,3,4563.0,88,0.145,8.729,60,...,277.7,307.7,246.9,70.750,6280,89,15.61,250,1602,30
4,Adam Schenk,28,239,0,1,5378.0,86,0.130,8.558,66,...,283.0,305.0,236.8,71.370,5683,80,16.88,243,1440,26


In [185]:
pgadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 69 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Player                     195 non-null    object 
 1   EVENTS_PLAYED              195 non-null    int64  
 2   POINTS                     195 non-null    int64  
 3   NUMBER_OF_WINS             195 non-null    int64  
 4   NUMBER_OF_TOP_Tens         195 non-null    int64  
 5   POINTS_BEHIND_LEAD         194 non-null    float64
 6   ROUNDS_PLAYED              195 non-null    int64  
 7   SG_PUTTING_PER_ROUND       195 non-null    float64
 8   TOTAL_SG:PUTTING           195 non-null    float64
 9   MEASURED_ROUNDS            195 non-null    int64  
 10  AVG_Driving_DISTANCE       195 non-null    float64
 11  UP_AND_DOWN_%              195 non-null    float64
 12  PAR_OR_BETTER              195 non-null    int64  
 13  MISSED_GIR                 195 non-null    int64  

In [186]:
pgadata = pgadata.drop(columns=['POINTS_BEHIND_LEAD', 'AGE', 'EVENTS_PLAYED', 'POINTS', 'NUMBER_OF_WINS', 'NUMBER_OF_TOP_Tens', 'ROUNDS_PLAYED',
                                'MEASURED_ROUNDS', 'HOLES_PLAYED', 'PAR_OR_BETTER', 'MISSED_GIR', 'FAIRWAYS_HIT', 'POSSIBLE_FAIRWAYS', 'GIR_RANK', 
                                'ATTEMPTS_GFG', 'NON-ATTEMPTS_GFG', 'NUMBER_OF_SAVES', 'NUMBER_OF_BUNKERS', 'TOTAL_DRIVES_FOR_320+', 'TOTAL_DRIVES', 
                                'TOTAL_STROKES', 'TOTAL_ROUNDS', 'BOGEYS_MADE'])

In [187]:
pgadata = pgadata.dropna()

In [188]:
pgadata_dex = pgadata.set_index('Player')
pgadata_dex.head(15)

,SG_PUTTING_PER_ROUND,TOTAL_SG:PUTTING,AVG_Driving_DISTANCE,UP_AND_DOWN_%,FAIRWAY_HIT_%,GOING_FOR_GREEN_IN_2%,RTP-GOING_FOR_THE_GREEN,RTP-NOT_GOING_FOR_THE_GRN,HOLE_OUTS,SAND_SAVE%,...,HIGHEST_SPIN_RATE,LOWEST_SPIN_RATE,AVG_HANG_TIME,LONGEST_ACT.HANG_TIME,SHORTEST_ACT.HANG_TIME,AVG_CARRY_DISTANCE,LONGEST_CARRY_DISTANCE,SHORTEST_CARRY_DISTANCE,AVG_SCORE,MAKES_BOGEY%
Player,,,,,,,,,,,,,,,,,,,,,
Aaron Baddeley,-0.062,-4.064,295.5,62.09,52.18,55.60,-86,-2,20,56.25,...,3761,2157,6.2,7.6,0.8,278.2,312.6,237.1,70.850,16.24
Aaron Wise,0.245,17.134,303.4,56.59,63.29,60.38,-97,-3,11,48.80,...,4507,1903,6.5,7.6,4.7,282.5,314.6,255.7,70.276,16.53
Abraham Ancer,-0.052,-4.076,296.3,63.23,63.06,59.87,-77,-18,13,54.38,...,6085,1700,6.4,7.6,4.6,275.1,305.8,223.5,70.659,14.76
Adam Hadwin,0.145,8.729,289.7,59.04,65.18,58.87,-68,-1,12,48.15,...,6446,2175,6.5,7.7,4.0,277.7,307.7,246.9,70.750,15.61
Adam Schenk,0.130,8.558,305.5,54.55,57.67,62.40,-81,4,8,42.36,...,3795,1913,6.5,7.7,4.0,283.0,305.0,236.8,71.370,16.88
Adam Scott,-0.221,-11.944,304.8,56.50,62.16,59.39,-68,-6,7,52.00,...,5255,2140,6.5,7.6,4.4,286.3,313.6,264.6,70.420,16.67
Alex Cejka,0.009,0.556,286.7,64.00,66.49,50.00,-64,-9,15,59.35,...,7586,2107,6.7,7.5,2.7,265.8,288.8,238.0,70.675,15.06
Alex Noren,0.686,28.117,299.2,61.96,60.52,67.72,-63,-1,4,51.58,...,6312,2105,6.7,7.9,2.9,280.9,308.1,260.2,69.973,15.09
Andrew Landry,-0.051,-3.516,291.7,56.14,63.23,45.91,-47,-25,15,42.28,...,5012,1456,6.3,7.3,2.2,272.2,296.0,244.2,71.059,16.53


In [189]:
pgadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Player                     195 non-null    object 
 1   SG_PUTTING_PER_ROUND       195 non-null    float64
 2   TOTAL_SG:PUTTING           195 non-null    float64
 3   AVG_Driving_DISTANCE       195 non-null    float64
 4   UP_AND_DOWN_%              195 non-null    float64
 5   FAIRWAY_HIT_%              195 non-null    float64
 6   GOING_FOR_GREEN_IN_2%      195 non-null    float64
 7   RTP-GOING_FOR_THE_GREEN    195 non-null    int64  
 8   RTP-NOT_GOING_FOR_THE_GRN  195 non-null    int64  
 9   HOLE_OUTS                  195 non-null    int64  
 10  SAND_SAVE%                 195 non-null    float64
 11  TOTAL_O/U_PAR              195 non-null    int64  
 12  Three_PUTT%                195 non-null    float64
 13  TOTAL_3_PUTTS              195 non-null    int64  

### Bryson DeChambeau

In [211]:
t_player = 'Bryson DeChambeau'
t_player_stat = pgadata_dex.loc[t_player]
dist = scipy.spatial.distance.cdist(pgadata_dex, [t_player_stat], metric = "euclidean")[:,0]
query_distances = list(zip(pgadata.index, dist))

In [212]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:11]:
    print(pgadata['Player'].iloc[sim_play], sim_stats)

Bryson DeChambeau 0.0
Chad Campbell 120.2345501551031
Keegan Bradley 161.61355041270517
Lanto Griffin 164.54087904530002
Shane Lowry 176.32901260144345
Patton Kizzire 185.05233692120726
Jason Dufner 191.58053334564036
Jonathan Byrd 207.5122951345293
Webb Simpson 211.43850619742852
Bud Cauley 226.0328650174572
Chez Reavie 253.94615328844816


### Justin Thomas

In [213]:
t_player = 'Justin Thomas'
t_player_stat = pgadata_dex.loc[t_player]
dist = scipy.spatial.distance.cdist(pgadata_dex, [t_player_stat], metric = "euclidean")[:,0]
query_distances = list(zip(pgadata.index, dist))

In [214]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:11]:
    print(pgadata['Player'].iloc[sim_play], sim_stats)

Justin Thomas 0.0
Wesley Bryan 166.5944385926493
Jamie Lovemark 214.03843075485293
John Huh 249.6198019248474
James Hahn 280.17895454334183
Whee Kim 280.6437076258793
Lanto Griffin 301.04364344227565
Patton Kizzire 301.15573469884333
Chad Campbell 327.5796357116846
Bryson DeChambeau 336.3875737657383
Jonathan Byrd 340.586173815086


### Tiger Woods

In [215]:
t_player = 'Tiger Woods'
t_player_stat = pgadata_dex.loc[t_player]
dist = scipy.spatial.distance.cdist(pgadata_dex, [t_player_stat], metric = "euclidean")[:,0]
query_distances = list(zip(pgadata.index, dist))

In [216]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:11]:
    print(pgadata['Player'].iloc[sim_play], sim_stats)

Tiger Woods 0.0
Chez Reavie 133.9262395686522
Matt Kuchar 136.42759957574566
Jimmy Walker 153.46735180812883
Bud Cauley 163.6686508131597
Webb Simpson 193.67082611999172
J.T. Poston 217.09077154268925
Derek Fathauer 248.62743523191486
Keegan Bradley 249.5107367389228
Rafa Cabrera Bello 270.3596863069641
Lanto Griffin 289.0309630368345


### Bubba Watson

In [217]:
t_player = 'Bubba Watson'
t_player_stat = pgadata_dex.loc[t_player]
dist = scipy.spatial.distance.cdist(pgadata_dex, [t_player_stat], metric = "euclidean")[:,0]
query_distances = list(zip(pgadata.index, dist))

In [218]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:11]:
    print(pgadata['Player'].iloc[sim_play], sim_stats)

Bubba Watson 0.0
Russell Knox 125.0577705422578
Talor Gooch 159.99427494132405
Billy Hurley III 181.21921728668838
Chris Stroud 203.86223783967446
Francesco Molinari 206.3080120790271
Sam Ryder 259.2002100404243
Marc Leishman 269.0862035575217
Kevin Na 275.4216755885419
Trey Mullinax 278.1370216511999
Hudson Swafford 281.65335429921646


### Rickie Fowler

In [219]:
t_player = 'Rickie Fowler'
t_player_stat = pgadata_dex.loc[t_player]
dist = scipy.spatial.distance.cdist(pgadata_dex, [t_player_stat], metric = "euclidean")[:,0]
query_distances = list(zip(pgadata.index, dist))

In [220]:
for sim_play, sim_stats in sorted(query_distances, key = lambda x: x[1], reverse = False)[:11]:
    print(pgadata['Player'].iloc[sim_play], sim_stats)

Rickie Fowler 0.0
Zac Blair 152.67668019052556
Rod Pampling 220.58789312652686
Charl Schwartzel 257.73684782739156
Keegan Bradley 267.1008483213786
Padraig Harrington 267.2992508612774
Bud Cauley 283.8479818652938
Webb Simpson 284.6081217534033
Andrew Yun 298.37170383767966
Russell Henley 312.0930388057382
Stewart Cink 315.5618963895989


: 